In [2]:
from bert.extract_features import InputExample
from bert import modeling, tokenization

In [3]:
import os 
import tensorflow as tf

In [4]:
os.chdir('/home/jovyan/at081-group39/python_code/Bert_Project/')

In [5]:
from data_preparation_for_feature_extraction import InputExample, loading_pickle_file, Inputdata_for_FE, convert_examples_to_features, input_fn_builder

In [6]:
layers = ['-1,-2,-3,-4'] 
extract_layers = layers
bert_config_file = 'chinese_L-12_H-768_A-12/bert_config.json'
vocab_file = 'chinese_L-12_H-768_A-12/vocab.txt'
init_checkpoint = 'chinese_L-12_H-768_A-12/bert_model.ckpt'
num_tpu_cores = 8
batch_size = 8

In [7]:
from Bert_preparation_function import  chinese_tokenizer

tokenizer = chinese_tokenizer()
all_news_path = '../../data_processed/news/all_news.p'
seq_length = 128
news_dict = loading_pickle_file(all_news_path)
# Bertinputdata_1 = Inputdata_for_FE(news_dict,text_type = 'content',method = 'single_sentence').Bert_Inputdata()
# Bertinputdata_1 = Inputdata_for_FE(news_dict,text_type = 'title',method = 'single_sentence').Bert_Inputdata()
Bertinputdata_1 = Inputdata_for_FE(news_dict,text_type = 'title'+'content',method = 'pair_sentence').Bert_Inputdata()
features = convert_examples_to_features(
                 examples=Bertinputdata_1, seq_length=seq_length, tokenizer=tokenizer)

unique_id_to_feature = {}
for feature in features:
    unique_id_to_feature[feature.unique_id] = feature
    input_fn = input_fn_builder(
                 features=features, seq_length=seq_length)

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] 三 大 電 腦 物 流 業 者 去 年 倒 帳 損 失 超 過 1 億 [SEP] 1998 年 由 於 景 氣 低 迷 , 連 帶 倒 帳 風 險 也 大 為 提 高 , 國 內 三 大 物 流 商 聯 強 、 捷 元 、 展 碁 去 年 倒 帳 金 額 , 合 計 已 經 逾 新 台 幣 1 億 元 , 由 於 物 流 業 毛 利 率 在 5 % 以 下 , 淨 利 率 更 低 於 1 % , 風 險 損 失 金 額 逼 近 此 一 比 例 的 業 者 , 將 難 以 立 足 市 場 。 [SEP]
INFO:tensorflow:input_ids: 101 676 1920 7442 5582 4289 3837 3511 5442 1343 2399 948 2379 3010 1927 6631 6882 122 1023 102 8368 2399 4507 3176 3250 3706 856 6837 117 6865 2380 948 2379 7591 7402 738 1920 4158 2990 7770 117 1751 1058 676 1920 4289 3837 1555 5474 2485 510 2949 1039 510 2245 4805 1343 2399 948 2379 7032 7540 117 1394 6243 2347 5195 6874 3173 1378 2395 122 1023 1039 117 4507 3176 4289 3837 3511 3688 1164 4372 1762 126 110 809 678 117 3912 1164 4372 3291 856 3176 122 110 117 7591 7402 3010 1927 7032 7540 6873 6818 3634 671 3683 891 4638 3511 5442 117 2200 7432 809 4989 6639 2356 1842 511 102 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 

In [8]:
def model_fn_builder(bert_config, init_checkpoint, layer_indexes, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    unique_ids = features["unique_ids"]
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    input_type_ids = features["input_type_ids"]

    model = modeling.BertModel(
        config=bert_config,
        is_training=False,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=input_type_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    if mode != tf.estimator.ModeKeys.PREDICT:
      raise ValueError("Only PREDICT modes are supported: %s" % (mode))

    tvars = tf.trainable_variables()
    scaffold_fn = None
    (assignment_map,
     initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(
         tvars, init_checkpoint)
#     TPU
    if use_tpu:

      def tpu_scaffold():
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
        return tf.train.Scaffold()

      scaffold_fn = tpu_scaffold
    else:
      tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    all_layers = model.get_all_encoder_layers()

    predictions = {
        "unique_id": unique_ids,
    }

    for (i, layer_index) in enumerate(layer_indexes):
      predictions["layer_output_%d" % i] = all_layers[layer_index]

    output_spec = tf.contrib.tpu.TPUEstimatorSpec(
        mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn

In [9]:
tf.logging.set_verbosity(tf.logging.INFO)
layer_indexes = [int(x) for x in layers[0].split(",")]
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

tokenizer = tokenization.FullTokenizer(
       vocab_file= vocab_file, do_lower_case=True)
#### tpu ####
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
      tpu_config=tf.contrib.tpu.TPUConfig(
          num_shards=num_tpu_cores,
          per_host_input_for_training=is_per_host))

#### tpu ####
model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=init_checkpoint,
      layer_indexes=layer_indexes,
      use_tpu=False,use_one_hot_embeddings=False)

estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=False,
      model_fn=model_fn,
      config=run_config,
      predict_batch_size=batch_size)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpxqg3p39v', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7ab13209e8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None), '_cluster': None}
INFO:tensorflow:_TPUContext: eval_on_tpu True


In [10]:
import codecs
import collections
import json
import numpy as np
with codecs.getwriter("utf-8")(tf.gfile.Open('output_title_content.jsonl',"w")) as writer:
    text_counter = 0                                                           #文章數量
    for result in estimator.predict(input_fn, yield_single_examples=True):     #每篇文章
#         print('article',result['unique_id'], news_dict[result['unique_id']]['new_id'])
        text_counter +=1                                                       #文章數量計算
        doc_vector = []                                                        #文章向量
        unique_id = int(result["unique_id"])
        feature = unique_id_to_feature[unique_id]
        output_json = collections.OrderedDict()
        output_json["linex_index"] = unique_id
        all_features = []
########## chia-hui modified ##########
        word_counter=0
        all_feature_extract = np.zeros([4,768])
        single_token_layer = []
        if unique_id%500 ==0:                                                 # Print 當前進度
            print(unique_id)
########## chia-hui modified ##########
        for (i, token) in enumerate(feature.tokens):                           #文章中每一個字
            all_layers = []
            word_counter+=1
            for (j, layer_index) in enumerate(layer_indexes):                  #前四層
                layer_output = result["layer_output_%d" % j]
                layers = collections.OrderedDict()
                layers["index"] = layer_index
                layers["values"] = [
                              round(float(x), 6) for x in layer_output[i:(i + 1)].flat]
                
########## chia-hui modified ##########
#                 print(word_counter,i,token,j,layer_index)
                if j==0:
                    single_token_layer.append(layers["values"])
                    for unit in range(768):
                        all_feature_extract[0][unit] = all_feature_extract[0][unit] + layers["values"][unit]
                elif j==1:
                    for unit in range(768):
                        all_feature_extract[1][unit] = all_feature_extract[1][unit] + layers["values"][unit]
                elif j==2:
                    for unit in range(768):
                        all_feature_extract[2][unit] = all_feature_extract[2][unit] + layers["values"][unit]
                else:
                    for unit in range(768):
                        all_feature_extract[3][unit] = all_feature_extract[3][unit] + layers["values"][unit]
########## chia-hui modified ##########

#                 all_layers.append(layers)
#             features = collections.OrderedDict()
#             features["token"] = token                                         #EX: "token": "[CLS]"
#             features["layers"] = all_layers                                   #EX: "layers": [{"index": -1, "values": [1.073043,...]}]
#             all_features.append(features)

########## chia-hui modified ##########
        all_feature_extract = (all_feature_extract/word_counter).round(5)                # 文章向量(4,768), 總和取字數平均
#         print('all_feature_extract',all_feature_extract)
        all_layer_feature = collections.OrderedDict()
        for layer_id in range(len(layer_indexes)):
            all_layer_feature[str(layer_indexes[layer_id])] = all_feature_extract[layer_id].tolist()
        doc_vector.append(all_layer_feature)
########## chia-hui modified ##########
#         print('doc_vector',doc_vector)
#         output_json["features"] = all_features
        output_json["new_id"] = news_dict[result['unique_id']]['new_id']
        output_json["doc_vector"] = doc_vector
        writer.write(json.dumps(output_json) + "\n")
    print('文章數',text_counter)

INFO:tensorflow:Could not find trained model in model_dir: /tmp/tmpxqg3p39v, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (21128, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/ke

In [ ]:
with open(,'wb') as file:
            pickle.dump(,file) 

In [21]:
with open("output_title.jsonl",'r') as file:
    for line in file.readlines():
        dic = json.loads(line)
        print(dic)

SyntaxError: can't assign to operator (<ipython-input-21-390f0844bff8>, line 1)

In [17]:
python extract_features.py \
  --input_file=/tmp/input.txt \
  --output_file=/tmp/output.jsonl \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --layers=-1,-2,-3,-4 \
  --max_seq_length=128 \
  --batch_size=8

SyntaxError: invalid syntax (<ipython-input-17-eab7b8107d95>, line 1)

In [20]:
news_dict[0]['date']
len(news_dict)

48503

In [32]:
news_dict[0:5]

[{'date': '1999-02-10',
  'new_id': '0b7a5750-7bcd-11d3-98ff-00e018a00403',
  'title': '三大電腦物流業者去年倒帳損失超過1億',
  'class': ['C023350'],
  'create_date': '1999-02-10 00:00:37',
  'content': '1998年由於景氣低迷, 連帶倒帳風險也大為提高, 國內三大物流商聯強、捷元、展碁去年倒帳金額, 合計已經逾新台幣1 億元, 由於物流業毛利率在5%以下, 淨利率更低於1%, 風險損失金額逼近此一比例的業者, 將難以立足市場。'},
 {'date': '1999-06-02',
  'new_id': '0b7a5b20-7bcd-11d3-98ff-00e018a00403',
  'title': '晶圓代工市場發燒台積電買德碁多一座八吋廠',
  'class': ['C023180'],
  'create_date': '1999-06-02 10:34:31',
  'content': '台積電將出資金額應在新台幣80億元以上，在近期內宣佈購入德碁半導體公司三成股權，德碁也將成為台積電另一個晶圓代工的場所。股權改組後的德碁，原本最大單一法人股東宏電，持股將由目前的49.3%降至30%，而德碁也將於88年第2季進行50億至60億元的現金增資，實收金額暫訂為100億元，年底直接申請第三類股票上市。目前德碁資本額為208億元，最大股東為宏碁集團，包括宏電、明電、宏科等集團事業單位，持股合計近58%，其中宏電持股便高達49.3%。<SEP>台積電預計拿下的股權，大部分是由宏電釋出，另有中華開發等原股東釋股。由於86、87年德碁虧損金額均超過50億元，88年下半年德碁準備辦理為85年發行的特別股部份減資16億元，將資本額降至192億元，再增資50億至60億元，台積電並將參與認購，以溢價15元至20元之間買入，準備將資本額提高至250億元至270億元，在完成下半年減資、增資後股本之後，總計台積電將持有德碁的股數在7億5,000萬股至8億1,000萬股之間，若以每股面值10元計算，台積電出資的金額將達80億元左右。<SEP>由於德碁過去一直在和IBM談引進晶圓代工技術引進問題，可是長久以來都沒見結